In [1]:
def str_to_vec(str_word_vec):
    str_word_vec = str_word_vec[12:-33]
    str_word_vec = re.sub(r'\[', '', str_word_vec)
    str_word_vec = re.sub(r'\]', '', str_word_vec)
    str_word_vec = re.sub(r'\n', ' ', str_word_vec)
    str_word_vec = re.sub(r'\s+', ' ', str_word_vec)
    vec = np.fromstring(str_word_vec, dtype=float, sep=' ')
    return vec

def str_to_vec_rep(str_word_vec):
    str_word_vec = str_word_vec[12:-33]
    str_word_vec = re.sub(r'\[', '', str_word_vec)
    str_word_vec = re.sub(r'\]', '', str_word_vec)
    str_word_vec = re.sub(r'\n', ' ', str_word_vec)
    str_word_vec = re.sub(r'\s+', ' ', str_word_vec)
    vec = np.fromstring(str_word_vec, dtype=float, sep=' ')
    return vec

def str_to_vec_rep_old(str_word_vec):
    str_word_vec = str_word_vec[79:-25]
    str_word_vec = re.sub(r'\[', '', str_word_vec)
    str_word_vec = re.sub(r'\]', '', str_word_vec)
    str_word_vec = re.sub(r'\n', ' ', str_word_vec)
    str_word_vec = re.sub(r'\s+', ' ', str_word_vec)
    vec = np.fromstring(str_word_vec, dtype=float, sep=',')
    return vec

In [2]:
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score
import pandas as pd

In [7]:
## LOAD DATA ###########################################################
import pandas as pd
import numpy as np
import re

# pos = pd.read_csv('pos_yago_7.csv')
# neg = pd.read_csv('neg_yago_7.csv')

# pos = pd.read_csv('pos_7.csv')
# neg = pd.read_csv('neg_7.csv')

# pos_unseen = pd.read_csv('zero_shot_pov_pairs.csv')
# neg_unseen = pd.read_csv('zero_shot_neg_pairs.csv')

# pos_train = pd.read_csv('zero_shot_pos_pairs_train.csv')
# neg_train = pd.read_csv('zero_shot_neg_pairs_train.csv')

# pos_yago = pd.read_csv('pos_yago_7.csv')
# neg_yago = pd.read_csv('neg_yago_7.csv')

# ###############################################
# pos_tac = pd.read_csv('pos_yago_7.csv')
# neg_tac = pd.read_csv('neg_yago_7.csv')

# pos_train_yago = pd.read_csv("tac_pos_train.csv")
# pos_test_yago = pd.read_csv("tac_pos_test.csv")

# neg_train_yago = pd.read_csv("tac_neg_train.csv")
# neg_test_yago = pd.read_csv("tac_neg_test.csv")

# pos_train_tac = pos_tac
# neg_train_tac = neg_tac
# ###############################################

# pos_tac = pd.read_csv('pos_yago_7.csv')
# neg_tac = pd.read_csv('neg_yago_7.csv')

# pos_train_yago = pd.read_csv("tac_pos_train.csv")
# pos_test_yago = pd.read_csv("tac_pos_test.csv")

# neg_train_yago = pd.read_csv("tac_neg_train.csv")
# neg_test_yago = pd.read_csv("tac_neg_test.csv")

# pos_train_tac = pos_tac
# neg_train_tac = neg_tac

# pos_unseen = pos_yago
# neg_unseen = neg_yago

pos_train = pd.read_csv('pos_7.csv')
neg_train = pd.read_csv('neg_7.csv')

pos_test = pd.read_csv('pos_yago_7.csv')
neg_test = pd.read_csv('neg_yago_7.csv')



############################################### GENERATE VECTORS #######################################################
from collections import defaultdict

yago_paper_step_1_with_triv = pd.DataFrame()
yago_paper_step_1_non_triv = pd.DataFrame()

auc_lst = []
err_lst = []
precision_lst = []
recall_last = []
f1_lst = []

agg_metrics_with_triv = defaultdict(list)
agg_metrics_non_triv = defaultdict(list)

for prct in [0.999, 0.99, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]:
    with_trivial = defaultdict(list)
    non_trivial = defaultdict(list)
    for i in range(1, 11):
        final_vecs_pos_train = []
        final_vecs_neg_train = []
        pos_train_labels = []
        neg_train_labels = []

        for index, row in pos_train.iterrows():
            diff_vec = []
            a_word_vec = str_to_vec_rep(row['a_word_vec'])
            b_word_vec = str_to_vec_rep(row['b_word_vec'])

            a_sentence_vec = str_to_vec_rep(row['a_sen_vec'])
            b_sentence_vec = str_to_vec_rep(row['b_sen_vec'])

            a_sentence_rep_vec = str_to_vec_rep(row['embed_replaced_sent_a'])
            b_sentence_rep_vec = str_to_vec_rep(row['embed_replaced_sent_b'])

            diff_vec.append(np.concatenate([(b_word_vec - a_word_vec) ** 2, 
                                            (b_sentence_vec - a_sentence_vec) ** 2,
                                            (b_sentence_rep_vec - a_sentence_rep_vec) ** 2,
                                            (a_sentence_rep_vec - a_sentence_vec) ** 2,
                                            (b_sentence_rep_vec - b_sentence_vec) ** 2]))

            diff_vec.append(row['edit_distance'])
            diff_vec.append(row['capitalized_edit_distance'])
            diff_vec.append(row['mention_b'])
            diff_vec.append(row['mention_a'])
            diff_vec.append(row['text_b'])
            diff_vec.append(row['text_a'])
            final_vecs_pos_train.append(diff_vec)
            pos_train_labels.append(1)

        for index, row in neg_train.iterrows():
                diff_vec = []
                a_word_vec = str_to_vec_rep(row['a_word_vec'])
                b_word_vec = str_to_vec_rep(row['b_word_vec'])

                a_sentence_vec = str_to_vec_rep(row['a_sen_vec'])
                b_sentence_vec = str_to_vec_rep(row['b_sen_vec'])

                a_sentence_rep_vec = str_to_vec_rep(row['embed_replaced_sent_a'])
                b_sentence_rep_vec = str_to_vec_rep(row['embed_replaced_sent_b'])

                diff_vec.append(np.concatenate([(b_word_vec - a_word_vec) ** 2, 
                                                (b_sentence_vec - a_sentence_vec) ** 2,
                                                (b_sentence_rep_vec - a_sentence_rep_vec) ** 2,
                                                (a_sentence_rep_vec - a_sentence_vec) ** 2,
                                                (b_sentence_rep_vec - b_sentence_vec) ** 2]))
                diff_vec.append(row['edit_distance'])
                diff_vec.append(row['capitalized_edit_distance'])
                diff_vec.append(row['mention_b'])
                diff_vec.append(row['mention_a'])
                diff_vec.append(row['text_b'])
                diff_vec.append(row['text_a'])
                final_vecs_neg_train.append(diff_vec)
                neg_train_labels.append(0)
        
        
        final_vecs_pos_test = []
        final_vecs_neg_test = []
        pos_labels_test = []
        neg_labels_test = []

        for index, row in pos_test.iterrows():
            diff_vec = []
            a_word_vec = str_to_vec_rep(row['a_word_vec'])
            b_word_vec = str_to_vec_rep(row['b_word_vec'])

            a_sentence_vec = str_to_vec_rep(row['a_sen_vec'])
            b_sentence_vec = str_to_vec_rep(row['b_sen_vec'])

            a_sentence_rep_vec = str_to_vec_rep(row['embed_replaced_sent_a'])
            b_sentence_rep_vec = str_to_vec_rep(row['embed_replaced_sent_b'])

            diff_vec.append(np.concatenate([(b_word_vec - a_word_vec) ** 2, 
                                            (b_sentence_vec - a_sentence_vec) ** 2,
                                            (b_sentence_rep_vec - a_sentence_rep_vec) ** 2,
                                            (a_sentence_rep_vec - a_sentence_vec) ** 2,
                                            (b_sentence_rep_vec - b_sentence_vec) ** 2]))

            diff_vec.append(row['edit_distance'])
            diff_vec.append(row['capitalized_edit_distance'])
            diff_vec.append(row['mention_b'])
            diff_vec.append(row['mention_a'])
            diff_vec.append(row['text_b'])
            diff_vec.append(row['text_a'])
            final_vecs_pos_test.append(diff_vec)
            pos_labels_test.append(1)

        for index, row in neg_test.iterrows():
            diff_vec = []
            a_word_vec = str_to_vec_rep(row['a_word_vec'])
            b_word_vec = str_to_vec_rep(row['b_word_vec'])

            a_sentence_vec = str_to_vec_rep(row['a_sen_vec'])
            b_sentence_vec = str_to_vec_rep(row['b_sen_vec'])

            a_sentence_rep_vec = str_to_vec_rep(row['embed_replaced_sent_a'])
            b_sentence_rep_vec = str_to_vec_rep(row['embed_replaced_sent_b'])

            diff_vec.append(np.concatenate([(b_word_vec - a_word_vec) ** 2, 
                                            (b_sentence_vec - a_sentence_vec) ** 2,
                                            (b_sentence_rep_vec - a_sentence_rep_vec) ** 2,
                                            (a_sentence_rep_vec - a_sentence_vec) ** 2,
                                            (b_sentence_rep_vec - b_sentence_vec) ** 2]))
            diff_vec.append(row['edit_distance'])
            diff_vec.append(row['capitalized_edit_distance'])
            diff_vec.append(row['mention_b'])
            diff_vec.append(row['mention_a'])
            diff_vec.append(row['text_b'])
            diff_vec.append(row['text_a'])
            final_vecs_neg_test.append(diff_vec)
            neg_labels_test.append(0)

        
#         final_vecs_pos_train_tac = []
#         final_vecs_neg_train_tac = []
#         pos_labels_train_tac = []
#         neg_labels_train_tac = []

#         for index, row in pos_train_tac.iterrows():
#             diff_vec = []
#             a_word_vec = str_to_vec_rep(row['a_word_vec'])
#             b_word_vec = str_to_vec_rep(row['b_word_vec'])

#             a_sentence_vec = str_to_vec_rep(row['a_sen_vec'])
#             b_sentence_vec = str_to_vec_rep(row['b_sen_vec'])

#             a_sentence_rep_vec = str_to_vec_rep(row['embed_replaced_sent_a'])
#             b_sentence_rep_vec = str_to_vec_rep(row['embed_replaced_sent_b'])

#             diff_vec.append(np.concatenate([(b_word_vec - a_word_vec) ** 2, 
#                                             (b_sentence_vec - a_sentence_vec) ** 2,
#                                             (b_sentence_rep_vec - a_sentence_rep_vec) ** 2,
#                                             (a_sentence_rep_vec - a_sentence_vec) ** 2,
#                                             (b_sentence_rep_vec - b_sentence_vec) ** 2]))

#             diff_vec.append(row['edit_distance'])
#             diff_vec.append(row['capitalized_edit_distance'])
#             diff_vec.append(row['mention_b'])
#             diff_vec.append(row['mention_a'])
#             diff_vec.append(row['text_b'])
#             diff_vec.append(row['text_a'])
#             final_vecs_pos_train_tac.append(diff_vec)
#             pos_labels_train_tac.append(1)


#         for index, row in neg_train_tac.iterrows():
#             diff_vec = []
#             a_word_vec = str_to_vec_rep(row['a_word_vec'])
#             b_word_vec = str_to_vec_rep(row['b_word_vec'])

#             a_sentence_vec = str_to_vec_rep(row['a_sen_vec'])
#             b_sentence_vec = str_to_vec_rep(row['b_sen_vec'])

#             a_sentence_rep_vec = str_to_vec_rep(row['embed_replaced_sent_a'])
#             b_sentence_rep_vec = str_to_vec_rep(row['embed_replaced_sent_b'])

#             diff_vec.append(np.concatenate([(b_word_vec - a_word_vec) ** 2, 
#                                             (b_sentence_vec - a_sentence_vec) ** 2,
#                                             (b_sentence_rep_vec - a_sentence_rep_vec) ** 2,
#                                             (a_sentence_rep_vec - a_sentence_vec) ** 2,
#                                             (b_sentence_rep_vec - b_sentence_vec) ** 2]))
#             diff_vec.append(row['edit_distance'])
#             diff_vec.append(row['capitalized_edit_distance'])
#             diff_vec.append(row['mention_b'])
#             diff_vec.append(row['mention_a'])
#             diff_vec.append(row['text_b'])
#             diff_vec.append(row['text_a'])
#             final_vecs_neg_train_tac.append(diff_vec)
#             neg_labels_train_tac.append(0)
        
        ############################################# TRAIN TEST SPLIT #########################################################


        from sklearn.model_selection import train_test_split
        
        # YAGO Train
        train_features_pos, _, train_labels_pos, _ = train_test_split(final_vecs_pos_train, pos_train_labels, test_size = 0.000001)
        train_features_neg, _, train_labels_neg, _ = train_test_split(final_vecs_neg_train, neg_train_labels, test_size = 0.000001)

        train_features_pos = list(train_features_pos)
        train_features_pos.extend(list(train_features_neg))
        train_features = train_features_pos
        
        train_labels_pos = list(train_labels_pos)
        train_labels_pos.extend(list(train_labels_neg))
        train_labels = train_labels_pos
        
#         val_features_pos = list(test_features_pos)
#         val_features_pos.extend(list(test_features_neg))
#         val_features = val_features_pos

#         val_labels_pos = list(test_labels_pos)
#         val_labels_pos.extend(list(test_labels_neg))
#         val_labels = np.array(val_labels_pos)
        
        train_features_pos_2, test_features_pos, train_labels_pos_2, test_labels_pos = train_test_split(final_vecs_pos_test, pos_labels_test, test_size = prct)
        train_features_neg_2, test_features_neg, train_labels_neg_2, test_labels_neg = train_test_split(final_vecs_neg_test, neg_labels_test, test_size = prct)
        
        train_features_pos_2 = list(train_features_pos_2)
        train_features_pos_2.extend(list(train_features_neg_2))
        train_features.extend(train_features_pos_2)
        
        train_labels_pos_2 = list(train_labels_pos_2)
        train_labels_pos_2.extend(list(train_labels_neg_2))
        train_labels.extend(train_labels_pos_2)
        train_labels = np.array(train_labels)
        
        test_features_pos = list(test_features_pos)
        test_features_pos.extend(list(test_features_neg))
        test_features = test_features_pos
        
        test_labels_pos = list(test_labels_pos)
        test_labels_pos.extend(list(test_labels_neg))
        test_labels = (np.array(test_labels_pos))
        
#         # TAC
#         train_features_pos_tac = final_vecs_pos_train_tac
#         train_features_pos_tac.extend(list(final_vecs_neg_train_tac))
#         train_features_tac = train_features_pos_tac

#         train_labels_pos_tac = pos_labels_train_tac
#         train_labels_pos_tac.extend(neg_labels_train_tac)
#         train_labels_tac = np.array(train_labels_pos_tac)

        
#         # YAGO Test
#         test_features_pos_unseen = final_vecs_pos_test_yago
#         test_features_pos_unseen.extend(final_vecs_neg_test_yago)
#         test_features_unseen = test_features_pos_unseen

#         test_labels_pos_unseen = pos_labels_test_yago
#         test_labels_pos_unseen.extend(neg_labels_test_yago)
#         test_labels_unseen = np.array(test_labels_pos_unseen)

        ############################################# ALL DATA RESULTS  ########################################################

        from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score
        import pandas as pd
        
        
#         params = {'loss_function':'Logloss',
#           'eval_metric': 'AUC', 
#           'verbose': 200,
#           'iterations': 100,
#           'learning_rate': 0.5, 
#          }
            
#         train_set = [np.append(np.append(t[0], t[1]), t[2]) for t in train_features_tac]
#         val_set = [np.append(np.append(t[0], t[1]), t[2]) for t in val_features]
        
#         model_base = CatBoostClassifier(**params)
#         model_base.fit(
#             train_set, 
#             train_labels_tac,
#             eval_set=(val_set, val_labels),
#             verbose=True, 
#         )
        
#         if prct != 0.999:
#             rf = CatBoostClassifier(**params)
#             train_unseen_set = [np.append(np.append(t[0], t[1]), t[2]) for t in train_features]
#             rf.fit(
#                 train_unseen_set, 
#                 train_labels,
#                 eval_set=(val_set, val_labels),
#                 init_model=model_base,
#                 verbose=True, 
#             )
#         elif prct == 0.999:
#             rf = model_base
    
        rf = RandomForestClassifier(n_estimators = 100)
        train_set = [np.append(np.append(t[0], t[1]), t[2]) for t in train_features]
        rf.fit(train_set, train_labels)
    
        predictions = rf.predict([np.append(np.append(t[0], t[1]), t[2]) for t in test_features])
        errors = abs(predictions - test_labels)

        roc_auc = roc_auc_score(test_labels, predictions)
        print('roc auc: ', roc_auc)
        print('error: ', sum(errors)/len(predictions))
        print('f1: ', f1_score(test_labels, predictions))
        print('precision :', precision_score(test_labels, predictions))
        print('recall :', recall_score(test_labels, predictions))  
        print('***********************')
        confusion_df = pd.DataFrame(confusion_matrix(test_labels, predictions))
        confusion_df.columns = ['predicted False', 'predicted True']
        confusion_df[''] = ['actual False', 'actual True']
        print(confusion_df.to_string(index=False))

        with_trivial['auc'].append(roc_auc)
        with_trivial['error'].append(sum(errors)/len(predictions))
        with_trivial['precision'].append(precision_score(test_labels, predictions))
        with_trivial['recall'].append(recall_score(test_labels, predictions))
        with_trivial['f1'].append(f1_score(test_labels, predictions))
        print("##################################################")
        
#         ############################################# NON TRIVIAL RESULTS  ########################################################
        try:
            from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score
            import pandas as pd
            
            test_features_not_trivial = []
            test_labels_not_trivial = []
            for i, t in enumerate(test_features):
                if (t[3] == t[4] and test_labels[i] == 0) or (t[3] != t[4] and test_labels[i] == 1):
                    test_features_not_trivial.append(t)
                    test_labels_not_trivial.append(test_labels[i])

            predictions = rf.predict([np.append(np.append(t[0], t[1]), t[2]) for t in test_features_not_trivial])
            errors = abs(predictions - test_labels_not_trivial)
            roc_auc = roc_auc_score(test_labels_not_trivial, predictions)

            non_trivial['auc'].append(roc_auc)
            non_trivial['error'].append(sum(errors)/len(predictions))
            non_trivial['precision'].append(precision_score(test_labels_not_trivial, predictions))
            non_trivial['recall'].append(recall_score(test_labels_not_trivial, predictions))
            non_trivial['f1'].append(f1_score(test_labels_not_trivial, predictions))

            print('roc auc: ', roc_auc)
            print('error: ', sum(errors)/len(predictions))
            print('f1: ', f1_score(test_labels_not_trivial, predictions))
            print('precision :', precision_score(test_labels_not_trivial, predictions))
            print('recall :', recall_score(test_labels_not_trivial, predictions))  
            print('***********************')
            confusion_df = pd.DataFrame(confusion_matrix(test_labels_not_trivial, predictions))
            confusion_df.columns = ['predicted False', 'predicted True']
            confusion_df[''] = ['actual False', 'actual True']
            print(confusion_df.to_string(index=False))
        except:
            continue


    ############################################# AGGREGATE RESULTS ################################################################


    import statistics
    
    print('# New Domain Positive Pairs Percentage - test: ' + str(prct))

    for key in with_trivial.keys():
        agg_metrics_with_triv[key].append(statistics.mean(with_trivial[key]))

    for key in non_trivial.keys():
        agg_metrics_non_triv[key].append(statistics.mean(non_trivial[key]))
                
    print(agg_metrics_with_triv)
    print(agg_metrics_non_triv)
############################################# SAVE CSVs ################################################################


yago_paper_step_1_with_triv['#_new_domain_pos_pairs_prct'] = [1, 0.99, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
yago_paper_step_1_with_triv['auc'] = agg_metrics_with_triv['auc']
yago_paper_step_1_with_triv['error'] = agg_metrics_with_triv['error']
yago_paper_step_1_with_triv['precision'] = agg_metrics_with_triv['precision']
yago_paper_step_1_with_triv['recall'] = agg_metrics_with_triv['recall']
yago_paper_step_1_with_triv['f1'] = agg_metrics_with_triv['f1']
yago_paper_step_1_with_triv.to_csv('Transfer_Learning_2_tac_to_yago_with_triv.csv')

yago_paper_step_1_non_triv['#_new_domain_pos_pairs_prct'] = [1, 0.99, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
yago_paper_step_1_non_triv['auc'] = agg_metrics_non_triv['auc']
yago_paper_step_1_non_triv['error'] = agg_metrics_non_triv['error']
yago_paper_step_1_non_triv['precision'] = agg_metrics_non_triv['precision']
yago_paper_step_1_non_triv['recall'] = agg_metrics_non_triv['recall']
yago_paper_step_1_non_triv['f1'] = agg_metrics_non_triv['f1']
yago_paper_step_1_non_triv.to_csv('Transfer_Learning_2_tac_to_yago_non_triv.csv')



<ipython-input-1-8f90cbef799c>:16: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  vec = np.fromstring(str_word_vec, dtype=float, sep=' ')


roc auc:  0.5809205907369364
error:  0.4178020996202814
f1:  0.6979410577311264
precision : 0.5475737995692386
recall : 0.9621549421193233
***********************
 predicted False  predicted True              
             891            3571  actual False
             170            4322   actual True
##################################################
roc auc:  0.5198017444312909
error:  0.4784164616416909
f1:  0.6746768060836502
precision : 0.5121219117986608
recall : 0.9884135472370766
***********************
 predicted False  predicted True              
             114            2113  actual False
              26            2218   actual True


<ipython-input-1-8f90cbef799c>:16: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  vec = np.fromstring(str_word_vec, dtype=float, sep=' ')


roc auc:  0.5506486355742546
error:  0.4479562206834934
f1:  0.6841483581384361
precision : 0.5293042524674059
recall : 0.9670525378450578
***********************
 predicted False  predicted True              
             599            3863  actual False
             148            4344   actual True
##################################################
roc auc:  0.5346147827625152
error:  0.46365466338626704
f1:  0.681811204911742
precision : 0.5200187309763521
recall : 0.9897504456327986
***********************
 predicted False  predicted True              
             177            2050  actual False
              23            2221   actual True


<ipython-input-1-8f90cbef799c>:16: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  vec = np.fromstring(str_word_vec, dtype=float, sep=' ')


...

roc auc:  0.8738404175988068
error:  0.12597547380156077
f1:  0.8809272918861959
precision : 0.8376753507014028
recall : 0.9288888888888889
***********************
 predicted False  predicted True              
             366              81  actual False
              32             418   actual True
##################################################
roc auc:  0.8005472350230416
error:  0.2018140589569161
f1:  0.8223552894211578
precision : 0.7253521126760564
recall : 0.9493087557603687
***********************
 predicted False  predicted True              
             146              78  actual False
              11             206   actual True


<ipython-input-1-8f90cbef799c>:16: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  vec = np.fromstring(str_word_vec, dtype=float, sep=' ')


roc auc:  0.8727293064876958
error:  0.12709030100334448
f1:  0.879746835443038
precision : 0.8373493975903614
recall : 0.9266666666666666
***********************
 predicted False  predicted True              
             366              81  actual False
              33             417   actual True
##################################################
roc auc:  0.8016195856873823
error:  0.2017353579175705
f1:  0.8201160541586073
precision : 0.726027397260274
recall : 0.9422222222222222
***********************
 predicted False  predicted True              
             156              80  actual False
              13             212   actual True


<ipython-input-1-8f90cbef799c>:16: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  vec = np.fromstring(str_word_vec, dtype=float, sep=' ')


roc auc:  0.8827964205816554
error:  0.11705685618729098
f1:  0.888178913738019
precision : 0.852760736196319
recall : 0.9266666666666666
***********************
 predicted False  predicted True              
             375              72  actual False
              33             417   actual True
##################################################
roc auc:  0.8096969696969696
error:  0.18859649122807018
f1:  0.8346153846153845
precision : 0.7508650519031141
recall : 0.9393939393939394
***********************
 predicted False  predicted True              
             153              72  actual False
              14             217   actual True
# New Domain Positive Pairs Percentage - test: 0.1
defaultdict(<class 'list'>, {'auc': [0.5723006097198347, 0.6048318807823172, 0.7643996474263228, 0.8193840398298143, 0.8373917033632148, 0.8543426031443966, 0.8588988689553176, 0.8660029370293078, 0.870548941725767, 0.8766331096196869, 0.8729612229679344], 'error': [0.42639044002680365, 0.